In [201]:
import pandas as pd
from scipy import stats
import numpy as np
from pandas.api.types import CategoricalDtype
import re

In [202]:
pl0 = pd.read_csv("../Kraftwerksliste_2020.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=9)
#pl0 = pd.read_csv("Kraftwerksliste_CSV.csv", error_bad_lines=False, engine="python")
bm = pd.read_csv("block_plant_mapper.csv", error_bad_lines=False, engine="python")

In [203]:
pl0.rename(columns={ pl0.columns[10]: "Energieträger_old", pl0.columns[13]: "Energieträger"}, inplace=True)

In [59]:
#pl0

In [60]:
#bm

In [61]:
#pl0.groupby("Energieträger").count()

In [62]:
#bm
pl1 = pl0.loc[np.logical_or(pl0.Energieträger == "Kernenergie", np.logical_or(pl0.Energieträger == "Erdgas", np.logical_or(pl0.Energieträger == "Steinkohle", np.logical_or(pl0.Energieträger == "Braunkohle", pl0.Energieträger == "Mineralölprodukte")))), :]

In [63]:
plt = pl1.dropna(subset=["Kraftwerksnummer Bundesnetzagentur"])

In [64]:
plt = plt.astype({"Energieträger": 'category'})

In [65]:
#plt["Energieträger"] = plt["Energieträger"].astype('category')
#plt["Energieträger"] = plt["Energieträger"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)

In [66]:
#plt["Energieträger"] = plt["Energieträger"].astype('category')

In [67]:
#plt.groupby('Energieträger').count()

In [68]:
pl2 = plt.loc[:]

In [69]:
#pl2.groupby("Energieträger").count()

In [70]:
# plt.dtypes

In [71]:
pl2 = plt.rename(columns={ pl1.columns[0]: "blockid", pl1.columns[3]: "PLZ", pl1.columns[4]: "Ort", pl1.columns[8]: "Inbetriebnahme", pl1.columns[9]: "Status", pl1.columns[15]: "KWK", pl1.columns[16]: "Nettoleistung"})
#pl3 = pl2.rename(columns={ pl1.columns[1]: "BlockID" })

In [72]:
def to_year(x):
    if type(x) == type(4):
        return x
    elif type(x) == type(2.5):
        return 0 # handle nans
    return int(x[-4:])

In [73]:
#blocks.loc[blocks.Energieträger == "Kernenergie"]

In [74]:
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('.','')
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(',','.')
pl2.loc[pl2.blockid == 'BNA0861a', 'Inbetriebnahme'] = 2012
pl2.loc[pl2.blockid == 'BNA1334', 'Inbetriebnahme'] = 2002
pl2.loc[pl2.blockid == 'BNA1141', 'Inbetriebnahme'] = 1970
pl2.loc[pl2.blockid == 'BNA0418', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.blockid == 'BNA1499', 'Inbetriebnahme'] = 1951
pl2.loc[pl2.blockid == 'BNA1502', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.blockid == 'BNA1500', 'Inbetriebnahme'] = 1990
pl2.loc[pl2.blockid == 'BNA1498', 'Inbetriebnahme'] = 1953
pl2.loc[pl2.blockid == 'BNA1260', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.blockid == 'BNA1056', 'Inbetriebnahme'] = 2006


#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('\n','')
#pl2['Nettoleistung'] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(';','.')

In [75]:
#pl2[] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
pl2['Inbetriebnahme'] = pl2['Inbetriebnahme'].apply(to_year)
pl2['Nettoleistung'] = pl2['Nettoleistung'].apply(pd.to_numeric)

In [76]:
#bm[pl2.duplicated(['BlockID'], keep=False)].sort_values("BlockID", ascending=False)

In [157]:
#pl2[pl2.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

In [169]:
cols = [10, 11,12,14,17,18,19]
pl3 = pl2.drop(pl2.columns[cols],axis=1)

In [177]:
#pl3.groupby('Unternehmen').max()

In [170]:
pl4 = pl3.copy()

In [180]:
def fix_company2(company):
    #print(company)
    company_dict = {"RWE": "RWE AG", "Vattenfall": "Vattenfall GmbH", "Uniper": "Uniper SE", "EnBW": "EnBW AG", "Steag": "Steag GmbH", "Nordzucker": "Nordzucker AG", "Lausitz Energie": "LEAG"}
    for key, value in company_dict.items():
        if key in str(company):
            return value
    return company

In [195]:
z = "endgülti2018"

In [198]:
def extract_still(x):
    match = re.findall("[0-9]{4}",x)
    return match[0] if match else np.nan

In [199]:
extract_still(z)

'2018'

In [189]:
#pl4 = pl3.ap

In [200]:
pl4['Unternehmen'] = pl4['Unternehmen'].apply(lambda x: fix_company2(x))
pl4['Stilllegung'] = pl4['Status'].apply(lambda x: extract_still(x))

In [179]:
#

In [80]:
# Status: endgültig stillgelegt, vorläufig stillgelegt, in Betrieb, Sicherheitsbereitschaft, Sonderfall, 
# Stillegung: [Jahreszahl]
# StA: ja/ nein

In [81]:
newdf = pd.DataFrame()

In [82]:
def generate_decom_tuple(state):
    if state == "In Betrieb":
        return("in Betrieb", "0")
    elif "Endgültig Stillgelegt" in state:
        a = "stillgelegt"
        b = str(int(state[22:26]))
        return (a, b)
    elif "Vorläufig Stillgelegt" in state:
        a = "vorläufig stillgelegt"
        return (a, "0")
    else:
        return(state, "0")

In [84]:
def fix_company(company):
    cmp = company
    # company_dict = {}
    company_dict = {"RWE": "RWE AG", "Vattenfall": "Vattenfall GmbH", "Uniper": "Uniper SE", "EnBW": "EnBW AG", "Steag": "Steag GmbH", "Nordzucker": "Nordzucker AG"}
    try:
        company_prefix, _ = company.split(" " , 1)
        cmp = company_dict[company_prefix]
    except (KeyError, AttributeError, ValueError):
        return company
    return cmp

In [85]:
for index, row in pl3.iterrows():
    #if row['Status'] == "Endgültig Stillgelegt 2016 (mit StA)":
    # print(row)
    if row['KWK'] == "nein":
        row['KWK'] = "Nein" # workaround for BoA plants
    status, stilllegung = generate_decom_tuple(row['Status'])
    cmp = fix_company(row['Unternehmen'])
    row['Unternehmen'] = cmp
    row['Status'] = status
    row['Stilllegung'] = stilllegung
    newdf = newdf.append(row)

In [86]:
'''
for index, row in pl3.iterrows():
    #if row['Status'] == "Endgültig Stillgelegt 2016 (mit StA)":
    #print(row)
    if row['KWK'] == "nein":
        row['KWK'] = "Nein" # workaround for BoA plants
    if row['KWK'] == "ja":
        row['KWK'] = "Ja"
    status, stilllegung = generate_decom_tuple(row['Status'])
    try:
        #new_cmp = fix_company(row['Unternehmen'])
        print(row)
    #row['Unternehmen'] = new_cmp
    #row['Status'] = status
    #row['Stilllegung'] = stilllegung
    #newdf = newdf.append(row)
'''

'\nfor index, row in pl3.iterrows():\n    #if row[\'Status\'] == "Endgültig Stillgelegt 2016 (mit StA)":\n    #print(row)\n    if row[\'KWK\'] == "nein":\n        row[\'KWK\'] = "Nein" # workaround for BoA plants\n    if row[\'KWK\'] == "ja":\n        row[\'KWK\'] = "Ja"\n    status, stilllegung = generate_decom_tuple(row[\'Status\'])\n    try:\n        #new_cmp = fix_company(row[\'Unternehmen\'])\n        print(row)\n    #row[\'Unternehmen\'] = new_cmp\n    #row[\'Status\'] = status\n    #row[\'Stilllegung\'] = stilllegung\n    #newdf = newdf.append(row)\n'

In [87]:
# newdf['Stilllegung'] = newdf['Stilllegung'].astype(int)
#newdf['Stilllegung'] = newdf['Stilllegung'].apply(lambda x: int(x) if x == x else None)

In [88]:
newdf['Stilllegung'] = newdf['Stilllegung'].replace("0", np.nan)

In [89]:
newdf.dtypes

Blockname                                      object
Bundesland                                     object
Energieträger                                  object
Inbetriebnahme                                float64
KWK                                            object
Kraftwerksname                                 object
Nettoleistung                                 float64
Ort                                            object
PLZ                                            object
Status                                         object
Stilllegung                                    object
Straße und Hausnummer (Standort Kraftwerk)     object
Unternehmen                                    object
blockid                                        object
dtype: object

In [90]:
newdf["Status"] = newdf["Status"].astype('category')
newdf["Status"] = newdf["Status"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)

In [91]:
newdf["KWK"] = newdf["KWK"].astype('category')
newdf["KWK"] = newdf["KWK"].cat.set_categories(['Ja', 'Nein'], ordered=True)

In [92]:
atest = newdf.groupby("KWK")

In [93]:
newdf.groupby("Energieträger").count()

,Blockname,Bundesland,Inbetriebnahme,KWK,Kraftwerksname,Nettoleistung,Ort,PLZ,Status,Stilllegung,Straße und Hausnummer (Standort Kraftwerk),Unternehmen,blockid
Energieträger,,,,,,,,,,,,,
Braunkohle,69,77,77,74,76,77,77,77,77,17,26,77,77
Erdgas,216,315,315,299,304,315,315,315,312,35,172,312,315
Kernenergie,9,9,9,9,9,9,9,9,9,3,2,9,9
Mineralölprodukte,47,56,56,55,56,56,56,56,51,10,30,56,56
Steinkohle,107,120,120,108,119,120,120,120,112,43,75,120,120


In [94]:
atest.count()

,Blockname,Bundesland,Energieträger,Inbetriebnahme,Kraftwerksname,Nettoleistung,Ort,PLZ,Status,Stilllegung,Straße und Hausnummer (Standort Kraftwerk),Unternehmen,blockid
KWK,,,,,,,,,,,,,
Ja,280,380,380,380,367,380,380,380,377,53,216,377,380
Nein,149,165,165,165,165,165,165,165,153,52,61,165,165


In [95]:
pl3 = newdf

In [96]:
atest = newdf.groupby(["Unternehmen"]).count()
atest.sort_values("Inbetriebnahme", inplace=True, ascending=False)

In [97]:
#print(atest)

In [98]:
atest = pl3.groupby(["Status"]).count()
# atest

In [99]:
# pl3.rename(columns={ pl3.columns[10]: "Energieträger"}, inplace=True)

In [100]:
bm[bm.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

,plantid,blockid


In [101]:
#bm2 = bm.drop_duplicates()

In [102]:
pl4 = bm.merge(pl3, how="outer", on="blockid")

In [103]:
#pl4[pl4.duplicated(['BlocID'], keep=False)].sort_values("BlockID", ascending=False)

In [104]:
pl5 = pl4[pd.notnull(pl4['blockid'])]
#pl5 = pl4
pl6 = pl5[pd.notnull(pl5['Nettoleistung'])]

In [105]:
# pl6

In [106]:
pl6['plantid'] = pl6['plantid'].apply(lambda x: str(x).replace('/', '_'))

/opt/conda/envs/idp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [107]:
# pl7 = pl6.replace("/", "_")

In [108]:
#pl7 = pl6.apply(str.replace("/", "_"))

In [109]:
#pl6.loc[pl6.Kraftwerksname == "KW Mittelsbüren"]

In [110]:
pl6['Inbetriebnahme'] = pl6['Inbetriebnahme'].apply(pd.to_numeric)

/opt/conda/envs/idp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [111]:
blocks = pl6.copy()
stammdaten = pl6.copy()

In [112]:
blocks.sort_values("Inbetriebnahme", inplace=True, ascending=False)
# blocks

In [113]:
plants = blocks.loc[:]
plants = plants.dropna(subset=["plantid"])

In [114]:
#plants_d = plants.loc[plants.duplicated("KraftwerkID", keep=False)]
# plants_d

In [115]:
#plants_c = plants_d.groupby(["KraftwerkID"]).sum()
#plants_count = plants_d.groupby(["KraftwerkID"]).count()

In [116]:
#plants_f = plants_c.reset_index()

In [117]:
#plants_count = plants_count.loc[:, ["blockid"]]

In [118]:
#plants_count = plants_count.reset_index()
# plants_count 

In [119]:
#plants_f = plants_f.merge(plants_count, on="KraftwerkID", how="inner")
# plants_f

In [120]:
#plants[plants.duplicated(['KraftwerkID'], keep=False)].sort_values("BlockID", ascending=False)

In [121]:
plants["Energieträger"] = plants["Energieträger"].astype('category')
plants["Energieträger"] = plants["Energieträger"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)
plants["Status"] = plants["Status"].astype('category')
plants["Status"] = plants["Status"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)
plants["KWK"] = plants["KWK"].astype('category')
plants["KWK"] = plants["KWK"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants["Bundesland"] = plants["Bundesland"].astype('category')
plants["Bundesland"] = plants["Bundesland"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [122]:
p_grp = plants.groupby("plantid")

In [123]:
plants.groupby("Energieträger").max()

,plantid,blockid,Inbetriebnahme,KWK,Nettoleistung,Ort,PLZ,Status
Energieträger,,,,,,,,
Kernenergie,666-18,BNA0802,1989.0,Nein,1410.0,Philippsburg,97506,stillgelegt
Braunkohle,664-08,BNA1511a,2014.0,Nein,1060.0,Zülpich,9116,stillgelegt
Steinkohle,663-07,BNA1674,2015.0,Nein,875.0,Zolling,A-44655,stillgelegt
Erdgas,nan,BNa1947,2019.0,Nein,887.0,Zülpich,A-91052,stillgelegt
Mineralölprodukte,669-01,BNA1338,2011.0,Nein,415.0,Zolling,94469,stillgelegt


In [124]:
plants_a = pd.DataFrame()
for plantid, group in p_grp:
    #if not plantid == "06-05-300-9046797":
    #    continue
    entry = {}
    #print(entry)
    entry["plantid"] = plantid
    # entry["BlockID"] = group
    try:
        entry["plantname"] = group["Kraftwerksname"].value_counts().index[0]
    except IndexError:
        entry["plantname"] = np.NaN
    entry["federalstate"] = group["Bundesland"].value_counts().index[0]
    entry["energysource"] = group["Energieträger"].min()
    try:
        entry["chp"] = group["KWK"].min()
    except TypeError:
        #print("aneror")
        #print(plantid)
        #print(group['KWK'].count())
        entry["chp"] = ""
    entry["latestexpanded"] = group["Inbetriebnahme"].max()
    entry["initialop"] = group["Inbetriebnahme"].min()
    entry["totalpower"] = group["Nettoleistung"].sum()
    entry["state"] = group["Status"].min()
    entry["blockcount"] = group["blockid"].count()
    try:
        entry["company"] = group["Unternehmen"].value_counts().index[0]
    except IndexError:
        ;
    #print(entry)
    #break
    plants_a = plants_a.append(entry, ignore_index=True)

In [125]:
int_list = ['blockcount', 'initialop', 'latestexpanded']
for column in int_list:
    plants_a[column] = plants_a[column].astype(int)
#plant['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)

In [126]:
plants_a.dtypes

blockcount          int64
chp                object
company            object
energysource       object
federalstate       object
initialop           int64
latestexpanded      int64
plantid            object
plantname          object
state              object
totalpower        float64
dtype: object

In [127]:
plants_a["energysource"] = plants_a["energysource"].astype('category')
plants_a["energysource"] = plants_a["energysource"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)
plants_a["state"] = plants_a["state"].astype('category')
plants_a["state"] = plants_a["state"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)
plants_a["chp"] = plants_a["chp"].astype('category')
plants_a["chp"] = plants_a["chp"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants_a["federalstate"] = plants_a["federalstate"].astype('category')
plants_a["federalstate"] = plants_a["federalstate"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [128]:
plants_a.dtypes

blockcount           int64
chp               category
company             object
energysource      category
federalstate      category
initialop            int64
latestexpanded       int64
plantid             object
plantname           object
state             category
totalpower         float64
dtype: object

In [129]:
plants_a.loc[plants_a.plantid == "06-05-300-0326774"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
52,9,Ja,RWE AG,Braunkohle,Nordrhein-Westfalen,1963,2002,06-05-300-0326774,Niederaußem,in Betrieb,3656.0


In [130]:
plants_a.loc[plants_a.plantid == "06-00176010435"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
22,4,Ja,Uniper SE,Steinkohle,Hessen,1965,1992,06-00176010435,Staudinger,in Betrieb,1624.0


In [131]:
plants_a.loc[plants_a.plantid == "06-05-900-0865327"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
76,9,Nein,RWE AG,Steinkohle,Nordrhein-Westfalen,1973,1984,06-05-900-0865327,Gersteinwerk,in Betrieb,2011.0


In [132]:
#plants_a = plants.drop_duplicates(subset="KraftwerkID")

In [133]:
column_titles = ['plantid', 'plantname', 'federalstate','energysource', 'chp', 'latestexpanded', 'initialop', 'totalpower', 'state', 'blockcount', 'company']
plants_b = plants_a.reindex(columns=column_titles)

In [134]:
#plants_b

In [135]:
blocks.drop(['Ort', 'PLZ', 'Straße und Hausnummer (Standort Kraftwerk)'], axis=1, inplace=True)

In [136]:
blocks[blocks.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

,plantid,blockid,Blockname,Bundesland,Energieträger,Inbetriebnahme,KWK,Kraftwerksname,Nettoleistung,Status,Stilllegung,Unternehmen
535,06-11-01-1105604,BNA0085b,Moabit GT 7,Berlin,Mineralölprodukte,1971.0,Nein,Moabit,34.0,in Betrieb,NaN,Vattenfall GmbH
536,06-11-01-1105604,BNA0085b,Moabit GT 7,Berlin,Mineralölprodukte,1971.0,Nein,Moabit,17.0,stillgelegt,2018,Vattenfall GmbH
551,662-05,BNA0083,Wilmersdorf GT1,Berlin,Mineralölprodukte,1977.0,Ja,Wilmersdorf,92.0,stillgelegt,2018,Vattenfall GmbH
550,662-05,BNA0083,Wilmersdorf GT2 und GT3,Berlin,Mineralölprodukte,1977.0,Ja,Wilmersdorf,184.0,in Betrieb,NaN,Vattenfall GmbH


In [137]:
#blocks

In [138]:
plants_b.sort_values("initialop", ascending=False, inplace=True)

In [139]:
#drop_list = ['Blockname', 'KWK', 'Nettoleistung', 'Stilllegung', 'Straße und Hausnummer (Standort Kraftwerk)', 'PLZ', 'Ort', 'Nettoleistung']
#plants_g = plants_f.drop(drop_list, axis=1)

In [140]:
#plants_h = plants_g.rename(columns={"KraftwerkID": "plantid", "Gesamtleistung": "totalpower", "Blockzahl": "blockcount", "BlockID": "blockid", "Bundesland": "federalstate", "Energieträger": "energysource", "Inbetriebnahme": "latestexpanded", "Kraftwerksname": "plantname", "Status": "state", "Unternehmen": "company"})

In [141]:
plants_final = plants_b.loc[:]

In [142]:
plants_final.dtypes

plantid             object
plantname           object
federalstate      category
energysource      category
chp               category
latestexpanded       int64
initialop            int64
totalpower         float64
state             category
blockcount           int64
company             object
dtype: object

In [143]:
# plants_final

In [144]:
# stammdaten

In [145]:
# cols = [0,2,3,9,10,11,12]
drop_list = ['plantid', 'Blockname', 'Energieträger', 'Inbetriebnahme', 'KWK', 'Kraftwerksname', 'Nettoleistung', 'Status', 'Stilllegung', 'Unternehmen']
stammdaten.drop(drop_list, axis=1, inplace=True)

In [146]:
#stammdaten.to_csv("stammdaten.csv", index=False)

In [147]:
#plants_final.to_csv("plants.csv", index=False)

In [148]:
#blocks.to_csv("blocks_3.csv", index=False)

In [149]:
stammdaten.to_csv("stammdaten_nh.csv", index=False, header=False)
blocks.to_csv("blocks_nh.csv", index=False, header=False)
plants_final.to_csv("plants_nh.csv", index=False, header=False)

In [150]:
#bm2.to_csv("bpm.csv", index=False)

In [151]:
#blocks

In [152]:
blocks.groupby('Energieträger').count()

,plantid,blockid,Blockname,Bundesland,Inbetriebnahme,KWK,Kraftwerksname,Nettoleistung,Status,Stilllegung,Unternehmen
Energieträger,,,,,,,,,,,
Braunkohle,77,77,69,77,77,74,76,77,77,17,77
Erdgas,315,315,216,315,315,299,304,315,312,35,312
Kernenergie,9,9,9,9,9,9,9,9,9,3,9
Mineralölprodukte,56,56,47,56,56,55,56,56,51,10,56
Steinkohle,120,120,107,120,120,108,119,120,112,43,120


In [153]:
pl4 = blocks.loc[blocks['Energieträger'] == "Mineralölprodukte"]

In [154]:
#pl4

In [155]:
#pd.set_option('display.max_rows', None)

In [156]:
#pl4.loc[pl4['Energieträger'] == "Mineralölprodukte"].sort_values(["Bundesland", "Unternehmen", "Kraftwerksname"])